# BDP Conversational Bot

---

Based on multilingual bot model, i will try make it usefull for answer questions without context, and being able to talk with the users

The dataset that i would use for train and test this case will be QA in English and Spanish I have combined for this model TriviaQA dataset and SQuAD_es dataset from this sites: 
 - https://github.com/ccasimiro88/TranslateAlignRetrieve
 - http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz


## Prepare Dataset

Lets create and prepare the dataset for the model

In [1]:
import requests
import tarfile
import os
url = "http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz"
target_path = 'triviaqa-unfiltered.tar.gz'

#Download the file if it doesn't exist
if target_path not in os.listdir():
    print("Downloading file")
    # Dowload the file 
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        print("Saving file")
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())

    # Unzip the file
    with tarfile.open(target_path, "r:gz") as tar:
        tar.extractall(path=".")
else:
    print("File already exists")

File already exists


In [2]:
# create a  Train Dataframe from the json file with the variables that i need of TriviaQA
import os
import pandas as pd
import json

if 'triviaqa-unfiltered.pkl' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-train.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            data_to_df.append({

                'question': question,
                'answers': [answer]
            })
        # Create a DataFrame from the list of dictionaries
        df_tr = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        df_tr.to_pickle('triviaqa-unfiltered.pkl')
        # Visualizar las primeras filas del DataFrame
    else:
        print("Json has not the dxpected format")
else:
    df_tr = pd.read_pickle('triviaqa-unfiltered.pkl')
    
df_tr.head()
df_tr.dropna(inplace=True)

In [3]:
# create a  Test Dataframe from the json file with the variables that i need of TriviaQA

if 'test-triviaqa-unfiltered.pkl' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-dev.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            data_to_df.append({
                'question': question,
                'answers': [answer]
            })

        # Create a DataFrame from the list of dictionaries
        test_tr = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        test_tr.to_pickle('test-triviaqa-unfiltered.pkl')
        # Visualizar las primeras filas del DataFrame

    else:
        print("Json has not the dxpected format")
else:
    test_tr = pd.read_pickle('test-triviaqa-unfiltered.pkl')
test_tr.dropna(inplace=True)
test_tr.head()

,question,answers
0,Who was the man behind The Chipmunks?,[David Seville]
1,What star sign is Jamie Lee Curtis?,[Scorpio]
2,Which Lloyd Webber musical premiered in the US...,[Sunset Boulevard]
3,Who was the next British Prime Minister after ...,[Campbell-Bannerman]
4,Who had a 70s No 1 hit with Kiss You All Over?,[Exile]


In [4]:
# Download dataset if it doesn't exist
import os
from datasets import load_dataset

target_path = 'squad_es'

if target_path not in os.listdir():
        # Dowload the file showing the progress bar
    print("Downloading file")
    dataset = load_dataset("squad_es", 'v1.1.0')  
    dataset.save_to_disk(target_path)  
else:
    print("File already exists")

e:\BertBot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


File already exists


In [5]:
from datasets import load_from_disk
import pandas as pd

# Cargar el conjunto de datos desde el disco
if "squad_train_T5.pkl" not in os.listdir() or "squad_test_T5.pkl" not in os.listdir():
    dataset = load_from_disk('squad_es')

    # Convertir el conjunto de datos en DataFrame de Pandas
    # Asumiendo que quieres convertir la parte 'train' del conjunto de datos
    df_sq = pd.DataFrame(dataset['train'])
    test_sq = pd.DataFrame(dataset['validation'])

    # eliminate the columns that i dont need
    df_sq.pop('id')
    df_sq.pop('title')
    df_sq.pop('context')

    test_sq.pop('id')
    test_sq.pop('title')  
    test_sq.pop('context')

    
    df_sq['answers'] = df_sq['answers'].apply(lambda x: x['text'])
    test_sq['answers'] = test_sq['answers'].apply(lambda x: x['text'])


    df_sq.to_pickle('squad_train_T5.pkl')
    test_sq.to_pickle('squad_test_T5.pkl')
    

else:
    df_sq = pd.read_pickle('squad_train_T5.pkl')
    test_sq = pd.read_pickle('squad_test_T5.pkl')
    


In [6]:
df_sq.dropna(inplace=True)
test_sq.dropna(inplace=True)

In [7]:
# Now, combine the two dataframes in one, mix the dataframes

if 'train_T5.pkl' not in os.listdir() or 'test_T5.pkl' not in os.listdir():
    df = pd.concat([df_tr, df_sq], ignore_index=True)
    test = pd.concat([test_tr, test_sq], ignore_index=True)
else:
    df = pd.read_csv('train_T5.pkl')
    test = pd.read_csv('test_T5.pkl')


In [8]:
df.dropna(inplace=True)
test.dropna(inplace=True)

In [9]:

from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Verify if Cuda is available to use GPU
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

# use multilingual cased model and its tokenizer
tokenizer_cased = T5Tokenizer.from_pretrained('T5-small')

CUDA (GPU) está disponible en tu sistema.


tokenizer_config.json: 100%|██████████| 2.32k/2.32k [00:00<00:00, 1.47MB/s]
e:\BertBot\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\carlo\.cache\huggingface\hub\models--T5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 2.00MB/s]

In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small") if not os.path.exists('./best_T5.pth') else T5ForConditionalGeneration.from_pretrained("./best_T5.pth")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100%|██████████| 1.21k/1.21k [00:00<00:00, 603kB/s]
model.safetensors: 100%|██████████| 242M/242M [00:04<00:00, 58.2MB/s] 
generation_config.json: 100%|██████████| 147/147 [00:00<00:00, 147kB/s]


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [12]:
def prepare_data_for_training(df):
    # prepare 
    inputs = ["Question: " + str(q) for q in df['question']]
    targets = [str(a) for a in df['answers']]
    return inputs, targets

# Extract Questions and Answers
train_inputs, train_targets = prepare_data_for_training(df)
test_inputs, test_targets = prepare_data_for_training(test)


In [13]:
def tokenize(inputs, targets):
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

if not os.path.exists('train_tokenized.pkl'):
    print("Tokenizing the text")

    # tokenize the information
    df_tokenized_data = tokenize(train_inputs, train_targets)
    test_tokenized_data = tokenize(test_inputs, test_targets)

    # Create the df
    df_tokenized = pd.DataFrame({key: pd.Series(value) for key, value in df_tokenized_data.items()})
    test_tokenized = pd.DataFrame({key: pd.Series(value) for key, value in test_tokenized_data.items()})

    # Save as pickle
    df_tokenized.to_pickle('train_tokenized.pkl')
    test_tokenized.to_pickle('test_tokenized.pkl')
else:
    print("Loading the tokenized text")
    df_tokenized = pd.read_pickle('train_tokenized.pkl')
    test_tokenized = pd.read_pickle('test_tokenized.pkl')

df_tokenized.head()

Tokenizing the text


e:\BertBot\.venv\lib\site-packages\transformers\tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


,input_ids,attention_mask,labels
0,"[11860, 10, 2645, 47, 1661, 116, 8, 166, 30520...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[784, 31, 15537, 651, 7953, 348, 31, 908, 1, 0..."
1,"[11860, 10, 4073, 797, 18, 7473, 8271, 7997, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[784, 31, 134, 77, 7997, 9765, 31, 908, 1, 0, ..."
2,"[11860, 10, 2840, 16, 2789, 47, 3, 17084, 446,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[784, 31, 476, 127, 157, 31, 908, 1, 0, 0, 0, ..."
3,"[11860, 10, 4599, 2144, 35, 7, 35, 13, 16168, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[784, 31, 2703, 49, 1072, 7, 31, 908, 1, 0, 0,..."
4,"[11860, 10, 86, 84, 5112, 410, 3259, 1976, 383...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[784, 31, 1458, 7, 31, 908, 1, 0, 0, 0, 0, 0, ..."


Create the datasets using Pytorch

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

class T5Dataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.dataframe.items()}
        return item


# Create the datasets for pytorch
train_dataset = T5Dataset(df_tokenized)
test_dataset = T5Dataset(test_tokenized)

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


## Train-Test 
---
After have complited the preprocessing phase, lets start to train and test our mode, for the moment we will use Adam optimizer, but we could try with another one

In [16]:

def train_test_T5(model, num_epochs, optimizer, train_loader, validation_loader, path_to_save='./best_T5.pth', max_loss=2000):
    for epoch in range(num_epochs):

        # Train Phase
        model.train()
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in validation_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_loss += loss.item()

        avg_loss = total_loss / len(validation_loader)

        print(f'Época {epoch}, Pérdida de Validación: {avg_loss}')

        # Save the model
        if avg_loss < max_loss:
            max_loss = avg_loss
            torch.save(model.state_dict(), path_to_save)
            print(f'Model Saved into {path_to_save} with this loss: {avg_loss}')


In [17]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# i have not enought memory to continue with the model
train_test_T5(model, 10, optimizer, train_loader, test_loader, path_to_save='./best_T5.pth')


e:\BertBot\.venv\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 12.00 GiB of which 0 bytes is free. Of the allocated memory 18.49 GiB is allocated by PyTorch, and 62.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF